In [1]:
import pandas as pd
import numpy as np
import difflib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import math

In [2]:
path = '/home/marcussilva/Documents/bases/'

## Lendo a base de dados

In [3]:
dt = pd.read_csv(path + 'dadosGenéricos.csv')

dt.head(5)

,CD_CONVENIO,NM_CONVENIO,CD_AVISO_CIRURGIA,CD_CIRURGIA_AVISO,NOVO_CD_CIRURGIA,DS_CIRURGIA,SN_PRINCIPAL,CD_CID,DS_CID,NOVO_CD_ESPECIALIDADE,...,NOVO_CD_ESPECIE_MESTRE,DS_ESPECIE_MESTRE,NOVO_CD_CLASSE_MESTRE,DS_CLASSE_MESTRE,NOVO_CD_SUB_CLA_MESTRE,DS_SUB_CLA_MESTRE,DT_REALIZACAO,QT_MOVIMENTACAO,ANO_REALIZACAO,ANO_MES_REALIZACAO
0,COV379,Convenio 0,AVC776,CAV731,NCIR412,Cirurgia 0,S,CID994,CID 0,ESP371,...,ESPM584,Especie Mestre 0,CLAM988,Classe Mestre 0,SUBCLAM925,Subclasse Mestre 0,2020-08-01,47,2024,2024-01
1,COV839,Convenio 1,AVC517,CAV970,NCIR382,Cirurgia 1,N,CID787,CID 1,ESP885,...,ESPM375,Especie Mestre 1,CLAM629,Classe Mestre 1,SUBCLAM933,Subclasse Mestre 1,2020-07-13,9,2020,2024-05
2,COV828,Convenio 2,AVC618,CAV197,NCIR866,Cirurgia 2,S,CID549,CID 2,ESP522,...,ESPM206,Especie Mestre 2,CLAM158,Classe Mestre 2,SUBCLAM356,Subclasse Mestre 2,2022-01-11,23,2022,2022-04
3,COV536,Convenio 3,AVC669,CAV467,NCIR744,Cirurgia 3,N,CID511,CID 3,ESP187,...,ESPM214,Especie Mestre 3,CLAM380,Classe Mestre 3,SUBCLAM179,Subclasse Mestre 3,2023-06-27,85,2022,2024-07
4,COV607,Convenio 4,AVC435,CAV559,NCIR556,Cirurgia 4,S,CID618,CID 4,ESP219,...,ESPM116,Especie Mestre 4,CLAM249,Classe Mestre 4,SUBCLAM779,Subclasse Mestre 4,2020-03-12,22,2020,2022-01


In [4]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CD_CONVENIO             20 non-null     object
 1   NM_CONVENIO             20 non-null     object
 2   CD_AVISO_CIRURGIA       20 non-null     object
 3   CD_CIRURGIA_AVISO       20 non-null     object
 4   NOVO_CD_CIRURGIA        20 non-null     object
 5   DS_CIRURGIA             20 non-null     object
 6   SN_PRINCIPAL            20 non-null     object
 7   CD_CID                  20 non-null     object
 8   DS_CID                  20 non-null     object
 9   NOVO_CD_ESPECIALIDADE   20 non-null     object
 10  DS_ESPECIALID           20 non-null     object
 11  NOVO_CD_PRODUTO         20 non-null     object
 12  DS_PRODUTO              20 non-null     object
 13  NOVO_CD_ESPECIE         20 non-null     object
 14  DS_ESPECIE              20 non-null     object
 15  NOVO_CD_

## Preparando os Kits

In [5]:
columns = ['DS_CIRURGIA', 'DS_PRODUTO_MESTRE', 'CD_AVISO_CIRURGIA','DS_ESPECIE']
df = dt[columns].groupby(['DS_CIRURGIA', 'DS_PRODUTO_MESTRE','DS_ESPECIE']).agg(
    freq_m = pd.NamedAgg(column = 'CD_AVISO_CIRURGIA', aggfunc = pd.Series.nunique)).sort_values(by='freq_m', ascending = False).reset_index()

df.head(5)

,DS_CIRURGIA,DS_PRODUTO_MESTRE,DS_ESPECIE,freq_m
0,Cirurgia 0,Produto Mestre 0,Especie 0,1
1,Cirurgia 1,Produto Mestre 1,Especie 1,1
2,Cirurgia 8,Produto Mestre 8,Especie 8,1
3,Cirurgia 7,Produto Mestre 7,Especie 7,1
4,Cirurgia 6,Produto Mestre 6,Especie 6,1


In [6]:
freqs = dt.groupby('DS_CIRURGIA')['CD_AVISO_CIRURGIA'].nunique()
freqs_dict = freqs.to_dict()

columns = ['DS_CIRURGIA', 'NOVO_CD_CIRURGIA','NOVO_CD_PRODUTO_MESTRE','DS_PRODUTO_MESTRE',
            'CD_CID','DS_CID','DS_PRODUTO','DS_UNIDADE_REFERENCIA','DS_ESPECIALID','CD_AVISO_CIRURGIA',
            'QT_MOVIMENTACAO', 'DS_CLASSE', 'ANO_MES_REALIZACAO']
dt_selected = dt[columns]

agg_funcs = {
    'QT_MOVIMENTACAO': ['min', 'max', 'mean', 'median', 'std'],
    'CD_AVISO_CIRURGIA': pd.Series.nunique
}
statistics = dt_selected.groupby(['DS_CIRURGIA', 'NOVO_CD_CIRURGIA','CD_AVISO_CIRURGIA',
                                    'NOVO_CD_PRODUTO_MESTRE','CD_CID','DS_CID','DS_PRODUTO_MESTRE',
                                    'DS_PRODUTO','DS_UNIDADE_REFERENCIA', 'DS_ESPECIALID', 'ANO_MES_REALIZACAO']).agg(agg_funcs)
statistics.columns = ['min', 'max', 'mean', 'median', 'std', 'freq_prod']  
statistics = statistics.sort_values(by='freq_prod', ascending=False).reset_index()
statistics['freq_prod%'] = statistics['freq_prod'] / statistics['DS_CIRURGIA'].map(freqs_dict)
statistics['std'].fillna(0, inplace=True)
statistics['qt'] = statistics['median'] + round(statistics['std'])
statistics['qt'] = statistics[['max', 'qt']].min(axis=1)

aux = pd.merge(statistics, df, on=['DS_CIRURGIA', 'DS_PRODUTO_MESTRE'], how='left')
aux['freq_m%'] = aux['freq_m'] / aux['DS_CIRURGIA'].map(freqs_dict)
aux.sort_values(by='freq_m%', inplace=True)

aux.head(5)

,DS_CIRURGIA,NOVO_CD_CIRURGIA,CD_AVISO_CIRURGIA,NOVO_CD_PRODUTO_MESTRE,CD_CID,DS_CID,DS_PRODUTO_MESTRE,DS_PRODUTO,DS_UNIDADE_REFERENCIA,DS_ESPECIALID,...,max,mean,median,std,freq_prod,freq_prod%,qt,DS_ESPECIE,freq_m,freq_m%
0,Cirurgia 0,NCIR412,AVC776,PRODM781,CID994,CID 0,Produto Mestre 0,Produto 0,Unidade Ref 0,Especialidade 0,...,47,47.0,47.0,0.0,1,1.0,47.0,Especie 0,1,1.0
17,Cirurgia 11,NCIR229,AVC769,PRODM714,CID574,CID 11,Produto Mestre 11,Produto 11,Unidade Ref 11,Especialidade 11,...,84,84.0,84.0,0.0,1,1.0,84.0,Especie 11,1,1.0
16,Cirurgia 12,NCIR917,AVC946,PRODM685,CID306,CID 12,Produto Mestre 12,Produto 12,Unidade Ref 12,Especialidade 12,...,16,16.0,16.0,0.0,1,1.0,16.0,Especie 12,1,1.0
15,Cirurgia 13,NCIR956,AVC735,PRODM860,CID603,CID 13,Produto Mestre 13,Produto 13,Unidade Ref 13,Especialidade 13,...,30,30.0,30.0,0.0,1,1.0,30.0,Especie 13,1,1.0
14,Cirurgia 14,NCIR940,AVC481,PRODM115,CID649,CID 14,Produto Mestre 14,Produto 14,Unidade Ref 14,Especialidade 14,...,14,14.0,14.0,0.0,1,1.0,14.0,Especie 14,1,1.0


### Funções de Comparação

#### Esta função foi feita para comparar dois kits provenientes de um mesmo procedimento mas com CIDs diferentes

In [10]:
def ComparaKits(a,b1,b2,c,n1,n2):

  k1 = aux.loc[(a)&(b1)&(aux['freq_m%']>c)]
  k1.to_csv(path + f'/Kits e Bases/kit_{n1}.csv')
  dt1 = pd.read_csv(path + f'/Kits e Bases/kit_{n1}.csv')
  kit1 = dt1['key_value'].nunique()
  print('Número de itens do primeiro kit:',kit1)

  k2 = aux.loc[(a)&(b2)&(aux['freq_m%']>c)]
  k2.to_csv(path + f'/Kits e Bases/kit_{n2}.csv')
  dt2 = pd.read_csv(path + f'/Kits e Bases/kit_{n2}.csv')
  kit2 = dt2['key_value'].nunique()
  print('Número de itens do segundo kit:',kit2)

  inter = pd.merge(dt1, dt2, how = 'inner', on = 'key_value')
  inter.to_csv(path + f'/kits-interseção/kit_concat_{n1}_&_{n2}.csv')
  it = pd.read_csv(path + f'/kits-interseção/kit_concat_{n1}_&_{n2}.csv')
  intersection = it['key_value'].nunique()
  print('Número de itens da interseção:',intersection)

  if kit1 > kit2:
    result = intersection/kit1
    print('Similaridade via Interseção: ',100*np.around(result,3),'%')
  else:
    result = intersection/kit2
    print('Similaridade via Interseção: ',100*np.around(result,3),'%')

#### Estas duas funções comparavam dois kits, sendo um da base de dados utilizada na pesquisa e um fornecido pelo HU-UFSCAR

In [ ]:
def string_similarity(str1, str2):
    result =  difflib.SequenceMatcher(a=str1.upper(), b=str2.upper())
    return result.ratio()

In [ ]:
def Compara_Kits(a,c,nm,kit):

  kt = aux.loc[(a)&(c)]
  kt.to_csv(path + f'/Kits e Bases/kit_{nm}_ret.csv')

  dt1 = pd.read_csv(path + f'/Kits e Bases/kit_{nm}_ret.csv')

  dt2 = pd.read_csv(path + f'/kits-tratados/{kit}')

  r1 = dt1['DS_PRODUTO_MESTRE'].nunique()
  print('Quantidade de itens no kit da base de dados:',r1)

  r2 = dt2['DS_PRODUTO_MESTRE'].nunique()
  print('Quantidade de itens no kit da UFSCAR:',r2)

  column1 = dt1['DS_PRODUTO_MESTRE'].drop_duplicates()
  column2 = dt2['DS_PRODUTO_MESTRE'].drop_duplicates()

  coluna1 = column1.to_numpy()
  coluna2 = column2.to_numpy()

  maior = 0
  intersection = []
  i = 0

  while(i < len(coluna1)):
    for j in range(len(coluna2)):
      res = string_similarity(coluna1[i],coluna2[j])
      if res > maior:
        maior = res
    if maior >= 0.40:
      intersection.append(maior)
    maior = 0
    i += 1

  inter = len(intersection)
  print('Quantidade de itens na interseção:',inter)

  if r1 > r2:
    result = (inter/r1) * 100
    print('A similaridade entre os kits é de:',np.around(result,2),'%')
  else:
    result = (inter/r2) * 100
    print('A quantidade entre os kits é de:',np.around(result,2),'%')